### Load packages etc, and load the child-class of gmspython, "Abatement", and load "techdata_to_tree" which converts a technology catalog into production trees

In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
os.chdir(py['main'])
import abatement, sys, ShockFunction
os.chdir(curr)
data_folder = os.getcwd()+'\\Data'
gams_folder = data_folder + "\\..\\gamsmodels\\Main"
# pickle_path = 

The file_gams_py_gdb1.gdx is still active and was not deleted.


In [2]:
m = abatement.abate(pickle_path=gams_folder+"/gmspython_Abatement", work_folder=work_folder)

In [3]:
m.initialize_variables()

In [4]:
m.use_EOP = True

In [5]:
m.add_blocks()

In [6]:
m.mblocks

{'M_Abatement_EOP',
 'M_Abatement_emissionaccounts',
 'M_Abatement_simplesumU',
 'M_Abatement_simplesumX_EOP',
 'M_EOP_CU',
 'M_EOP_TU',
 'M_EOP_TX',
 'M_ID_CU',
 'M_ID_EC',
 'M_ID_IOCU',
 'M_ID_IOX',
 'M_ID_TU',
 'M_ID_TX',
 'M_ID_UbaseX'}

In [3]:
m.use_EOP = True

In [8]:
m.mblocks

TypeError: unhashable type: 'set'

In [6]:
m.mblocks

TypeError: unhashable type: 'set'

In [9]:
m.model.functions = {"std_pdf":"$FUNCTION std_pdf({x}): ((1/(sqrt(2*Pi)))*exp(-(Sqr({x}))/2)) $ENDFUNCTION"}

In [17]:
print(m.blocktext)

{'M_ID_EC': 'E_zp_out_ID_EC[n]$(ID_out_ID_EC[n])..\tPbT[n]*qS[n] =E= sum(nn$(map_ID_EC[nn,n]), qD[nn]*PwThat[nn]);\n\tE_zp_nout_ID_EC[n]$(kno_no_ID_EC[n])..\tPwThat[n]*qD[n] =E= sum(nn$(map_ID_EC[nn,n]), qD[nn]*PwThat[nn]);\n\tE_q_out_ID_EC[n]$(bra_o_ID_EC[n])..\tqD[n] =E= sum(nn$(map_ID_EC[n,nn]), mu[n,nn] * (PbT[nn]/PwThat[n])**(sigma[nn]) * qS[nn] / sum(nnn$(map_ID_EC[nnn,nn]), mu[nnn,nn] * (PbT[nn]/PwThat[nnn])**(sigma[nn])));\n\tE_q_nout_ID_EC[n]$(bra_no_ID_EC[n])..\tqD[n] =E= sum(nn$(map_ID_EC[n,nn]), mu[n,nn] * (PwThat[nn]/PwThat[n])**(sigma[nn]) * qD[nn] / sum(nnn$(map_ID_EC[nnn,nn]), mu[nnn,nn] * (PwThat[nn]/PwThat[nnn])**(sigma[nn])));', 'M_ID_CU': 'E_zp_out_ID_CU[n]$(ID_out_ID_CU[n])..\tPbT[n]*qS[n] =E= sum(nn$(map_ID_CU[nn,n]), qD[nn]*PwThat[nn]);\n\tE_zp_nout_ID_CU[n]$(kno_no_ID_CU[n])..\tPwThat[n]*qD[n] =E= sum(nn$(map_ID_CU[nn,n]), qD[nn]*PwThat[nn]);\n\tE_q_out_ID_CU[n]$(bra_o_ID_CU[n])..\tqD[n] =E= sum(nn$(map_ID_CU[n,nn]), mu[n,nn] * exp((PbT[nn]-PwThat[n])*sigma[nn])

In [11]:
m.ns_local

{'ID_EC': {'name': 'ID_EC',
  'type_io': 'input',
  'version': 'std',
  'n': 'n',
  'nn': 'nn',
  'nnn': 'nnn',
  'map_': 'map_ID_EC',
  'temp_namespace': None,
  'type_f': 'CES_norm',
  'tree_out': 'ID_out_ID_EC',
  'knots': 'kno_ID_EC',
  'kno_no': 'kno_no_ID_EC',
  'bra_o': 'bra_o_ID_EC',
  'bra_no': 'bra_no_ID_EC'},
 'ID_CU': {'name': 'ID_CU',
  'type_io': 'input',
  'version': 'std',
  'n': 'n',
  'nn': 'nn',
  'nnn': 'nnn',
  'map_': 'map_ID_CU',
  'temp_namespace': None,
  'type_f': 'MNL',
  'tree_out': 'ID_out_ID_CU',
  'knots': 'kno_ID_CU',
  'kno_no': 'kno_no_ID_CU',
  'bra_o': 'bra_o_ID_CU',
  'bra_no': 'bra_no_ID_CU'},
 'ID_TU': {'name': 'ID_TU',
  'type_io': 'output',
  'version': 'std',
  'n': 'n',
  'nn': 'nn',
  'nnn': 'nnn',
  'map_': 'map_ID_TU',
  'temp_namespace': None,
  'type_f': 'CET_norm',
  'tree_out': 'ID_out_ID_TU',
  'knots': 'kno_ID_TU',
  'kno_no': 'kno_no_ID_TU',
  'bra_o': 'bra_o_ID_TU',
  'bra_no': 'bra_no_ID_TU'},
 'ID_TX': {'name': 'ID_TX',
  'type_io

In [10]:
m.write_and_run()

KeyError: ('M_EOP_TU', ' is not a valid equation, block of equations, or model')

In [10]:
# m.model.settings.databases["Abatement_0"]["sigma"].vals.loc[:] = 0.0001
# m.model.settings.databases["Abatement_0"]["eta"].vals.loc[:] = -0.0001

m.write_and_run(kwargs_init={'check_variables':True})
if m.model_instances["baseline"].modelstat == 16.0:
    print("\nSuccess! The modelstat was 16.0")


Success! The modelstat was 16.0


#### Sneaky solve to adjust sigmas into desired values

The model could be solved using elasticities of 0.0001. Let's check if it can be solved when these are set to higher numbers, implying a solution farther from the starting values (which, as mentioned, were set based on Leontief).

In [9]:
# condition = m.model.settings.databases["Abatement"]["sigma"].vals.index.str.startswith("C_E") & ~m.model.settings.databases["Abatement"]["sigma"].vals.index.str.contains("base")

In [10]:
#Sigmas in MNL nests of end-of-pipe cannot be Leontief, because share-parameters are inconsistent with quantities, and so we need prices to compensate.


In [11]:
m.model.settings.databases

{'ID_0': <DataBase.GPM_database at 0x1d5e12b7b88>}

In [12]:
for sigma_val in range(0, 450, 10):
    sigma_val = sigma_val/100
    if sigma_val == 0:
        sigma_val = 0.0001
#     for MNL_sigma in [0.1, 1]:

    #Set values
    #m.model.settings.databases["Abatement"]["sigma"].vals.loc[condition] = MNL_sigma
    #m.model.settings.databases["Abatement"]["sigma"].vals.loc[~condition] = sigma_val
#     m.get("sigma")[[t for t in m.get("sigma").index if t not in m.get("EOP_C_subset")]] = sigma_val
    m.model.settings.databases["ID_0"]["sigma"].vals.loc[:] = sigma_val
    m.model.settings.databases["ID_0"]["eta"].vals.loc[:] = -sigma_val
#     m.get("sigma")[m.get("EOP_C_subset")] = sigma_val
#     m.get("eta")[m.get("eta").index.str.startswith("EOP")] = -sigma_val

    #Try to solve
    m.write_and_run()

    #How did it go?
    if m.model_instances["baseline"].modelstat == 16.0:
        print("\nSUCCESS:")
        print("Sigma/eta:", round(sigma_val, 4))
        #print("Sigma/eta (MNL):", round(MNL_val, 4))
    else:
        print("\nFAIL:")
        print("Sigma/eta:", round(sigma_val, 4))
        #print("Sigma/eta: (MNL)", round(MNL_sigma, 4))


FAIL:
Sigma/eta: 0.0001

FAIL:
Sigma/eta: 0.1

FAIL:
Sigma/eta: 0.2

FAIL:
Sigma/eta: 0.3

FAIL:
Sigma/eta: 0.4

FAIL:
Sigma/eta: 0.5

FAIL:
Sigma/eta: 0.6

FAIL:
Sigma/eta: 0.7

FAIL:
Sigma/eta: 0.8

FAIL:
Sigma/eta: 0.9

FAIL:
Sigma/eta: 1.0

FAIL:
Sigma/eta: 1.1

FAIL:
Sigma/eta: 1.2

FAIL:
Sigma/eta: 1.3

FAIL:
Sigma/eta: 1.4

FAIL:
Sigma/eta: 1.5

FAIL:
Sigma/eta: 1.6

FAIL:
Sigma/eta: 1.7

FAIL:
Sigma/eta: 1.8

FAIL:
Sigma/eta: 1.9

FAIL:
Sigma/eta: 2.0

FAIL:
Sigma/eta: 2.1

FAIL:
Sigma/eta: 2.2

FAIL:
Sigma/eta: 2.3

FAIL:
Sigma/eta: 2.4

FAIL:
Sigma/eta: 2.5

FAIL:
Sigma/eta: 2.6

FAIL:
Sigma/eta: 2.7

FAIL:
Sigma/eta: 2.8

FAIL:
Sigma/eta: 2.9

FAIL:
Sigma/eta: 3.0

FAIL:
Sigma/eta: 3.1

FAIL:
Sigma/eta: 3.2

FAIL:
Sigma/eta: 3.3

FAIL:
Sigma/eta: 3.4

FAIL:
Sigma/eta: 3.5

FAIL:
Sigma/eta: 3.6

FAIL:
Sigma/eta: 3.7

FAIL:
Sigma/eta: 3.8

FAIL:
Sigma/eta: 3.9

FAIL:
Sigma/eta: 4.0

FAIL:
Sigma/eta: 4.1

FAIL:
Sigma/eta: 4.2

FAIL:
Sigma/eta: 4.3

FAIL:
Sigma/eta: 4.4


As can be seen, the model cannot be solved when the elasticities get too large. To accomodate this, we replace each iteration's starting values with the previous iteration's solution, while gradually increasing $\sigma$ and $\mu$.
This allows us to set these to (hopefully) any value. This "gradual changing of parameters" is called 'sneaky solve'. :)

### Sneaky solve: Gradual increase in sigma

In [9]:
db_star = DataBase.GPM_database()

In [10]:
sigma_star = m.get("sigma").copy()
eta_star = m.get("eta").copy()

In [11]:
#Energy services combine their underlying components with mild substitution possibilities
sigma_star[m.get("kno_ID_EC")] = 0.5

In [14]:
#All CET splits get more substitution
eta_star[:] = -2

In [15]:
#The IO technology treats its outputs as perfect substitutes. This ensures they have the same price in equiliibrum.
eta_star["IO_tech"] = -10

In [16]:
#Components have a large degree of substitution between overlapping technology goods:
sigma_star[m.get("kno_ID_CU")] = 5

In [17]:
db_star["sigma"] = sigma_star
db_star["eta"] = eta_star

Run again, while adding a checkpoint that we can use afterwards

In [18]:
m.checkpoints

{}

In [19]:
m.model_instances

{'baseline': <DB2Gams_l3.gams_model at 0x22937c08348>}

In [20]:
name = "sneaky"
#Fejler med noget "delimiter" fejl
#m.write_and_run(name="test", add_checkpoint="test", options_add={"checkpoint":m.checkpoints["leontief"]})
m.write_and_run(name=name, add_checkpoint="leontief")

In [21]:
m.checkpoints[name] = m.model_instances[name].ws.add_checkpoint()

In [22]:
m.checkpoints

{'leontief': <gams.execution.GamsCheckpoint at 0x22937b25b88>,
 'sneaky': <gams.execution.GamsCheckpoint at 0x22937fa6248>}

Solve sneakily, gradually increasing sigmas towards their desired values (which themselves are stated in `db_star`)

In [23]:
m.model_instances[name].solve_sneakily(db_star=db_star, from_cp=True, cp_init=m.checkpoints["leontief"], options_run={"checkpoint":m.checkpoints[name]})

{'Modelstat': 16.0, 'Solvestat': 1.0}

In [24]:
mi = m.model_instances[name]
# newjob = mi.ws.add_job_from_string("sets a,b;",**{'checkpoint': m.checkpoints[name]})
# newjob.run()

Check if the model ran succesfully:

In [25]:
if m.model_instances[name].out_db.series["Abatement_B_modelstat"].vals == 16:
    print("Ran successfully with modelstat " + str(m.model_instances[name].out_db.series["Abatement_B_modelstat"].vals))

Ran successfully with modelstat 16.0


The changes in e.g. demanded quantities, from the initial point (the solution where sigmas were ~0) are:

In [26]:
m.model_instances[name].out_db["qD"].vals - m.get("qD")

n
C_EH_1                   1.018307
C_EH_2                  -0.086536
C_EH_3                  -0.168528
C_EH_base               -0.763243
C_EL_1                   0.279783
                           ...   
U_ID_C_EL_4_base         2.729239
U_ID_C_EL_4_base_inp4    2.729239
U_ID_C_EL_5_base         0.882456
U_ID_C_ER_1_base         0.945925
U_ID_C_ER_2_base         0.949993
Name: qD, Length: 99, dtype: float64

while demand itself ended in the following quantities:

In [27]:
m.model_instances[name].out_db["qD"].vals

n
U_ID_C_EL_1_base         5.279783e+00
ID_5_inp4                5.394925e-09
ID_2_electricity         8.015852e-11
U_ID_C_ER_2_base         3.949993e+00
U_ID_C_EH_1_base_inp3    1.601831e+01
                             ...     
EOP_t3_K                 3.700596e+02
U_EOP_t1_2               5.543436e+01
ID_1                     6.565749e-09
U_EOP_t2_2               1.000000e+02
C_EH_base                7.623676e+01
Name: qD, Length: 99, dtype: float64

And let's confirm that our sigmas actually ended in the desired spots:

In [28]:
m.model_instances[name].out_db["sigma"].vals

n
U_ID_C_EL_1_base    0.0001
C_SO2_1             2.0000
ID_5_inp4           0.0000
sum_inp4            0.0000
ER                  0.5000
                     ...  
U_EOP_t1_2          0.0000
ID_1                0.0001
CO2                 0.0000
U_EOP_t2_2          0.0000
C_EH_base           0.0000
Name: sigma, Length: 128, dtype: float64

In [29]:
db = m.model_instances['sneaky'].out_db
m.model.database.merge_dbs(m.model.database,db,'second')
m.reset_settings()

In [30]:
m.setstate("calibrate")

In [33]:
# m.model.settings.databases["Abatement_0"]["sigma"].vals.loc[:] = 0.0001
# m.model.settings.databases["Abatement_0"]["eta"].vals.loc[:] = -0.0001
#m.model.settings.databases["Abatement"]["sigma"].vals[condition] = 0.2
#m.write_and_run(options_run={'output':sys.stdout})
m.write_and_run(overwrite=True)
if m.model_instances["baseline"].modelstat == 2.0:
    print("\nNPL problem solved successfully! The modelstat was 2.0")
else:
    print("Failed.")

Failed.


## A shock to the oil price
We present graphically what happens when the oil price gradually increases (static model solution for each price).
For this, we again use the sneaky solve, but we make sure to extract variables of interest for each iteration, in contrast to above where we were not interested in the solutions per se, but simply that the values of $\sigma$ ended in the desired spots.

In [121]:
#mideleritid start fra starten
#m.model.settings.databases["Abatement_0"]["sigma"].vals.loc[:] = 0.1
#m.model.settings.databases["Abatement_0"]["eta"].vals.loc[:] = -0.1

In [122]:
# m.write_and_run(name="calibrated", add_checkpoint="calibrated")

In [123]:
m.model_instances["calibrated"].out_db.get("sigma")

n
C_CO2_1             0.50
C_EH_1              1.20
C_EH_2              1.20
C_EH_3              1.20
C_EH_base           0.01
C_EL_1              1.20
C_EL_2              1.20
C_EL_3              1.20
C_EL_4              1.20
C_EL_5              1.20
C_EL_base           0.01
C_ER_1              1.20
C_ER_2              1.20
C_ER_base           0.01
EH                  0.01
EL                  0.01
EOP_tau1            0.50
ER                  0.01
ID_1                0.01
ID_2                0.01
ID_3                0.01
ID_4                0.01
ID_5                0.01
U_ID_C_EH_1_base    0.01
U_ID_C_EH_2_base    0.01
U_ID_C_EH_3_base    0.01
U_ID_C_EL_1_base    0.01
U_ID_C_EL_2_base    0.01
U_ID_C_EL_3_base    0.01
U_ID_C_EL_4_base    0.01
U_ID_C_EL_5_base    0.01
U_ID_C_ER_1_base    0.01
U_ID_C_ER_2_base    0.01
Name: sigma, dtype: float64

In [124]:
out_db = m.model_instances["calibrated"].out_db

In [125]:
m.write_and_run(name="oilshock")
#db_out = m.model_instances["baseline"].out_db

In [126]:
#m.write_and_run(name='taxshock', add_checkpoint='taxshock', overwrite=True)
mi = m.model_instances['oilshock']

In [127]:
#mi.out_db.series.__dict__

*2: Define structure of shock* 

In [128]:
db_end_of_loop = DataBase.GPM_database()

In [129]:
cond = out_db.get("PwT").index.str.contains("oil")
db_end_of_loop["PwT"] = out_db.get("PwT")[cond] + 12
#end_prices[cond] = out_db.get("PwT")[cond] + 5
#end_prices = end_prices[cond]

In [130]:
db_end_of_loop.series["PwT"].vals

n
C_EH_base_oil           13.0
C_EL_base_oil           13.0
C_ER_base_oil           13.0
ID_1_oil                13.0
ID_2_oil                13.0
ID_5_oil                13.0
U_ID_C_EH_1_base_oil    13.0
U_ID_C_EH_2_base_oil    13.0
U_ID_C_EH_3_base_oil    13.0
U_ID_C_EL_1_base_oil    13.0
U_ID_C_EL_2_base_oil    13.0
U_ID_C_EL_3_base_oil    13.0
U_ID_C_EL_4_base_oil    13.0
U_ID_C_EL_5_base_oil    13.0
U_ID_C_ER_1_base_oil    13.0
U_ID_C_ER_2_base_oil    13.0
Name: PwT, dtype: float64

In [131]:
#NY ME DDIREKTE
#m.model_instances["oilshock"].solve_sneakily(db_star=db_end_of_loop, from_cp=True, cp_init=m.checkpoints["calibrated"])

In [132]:
(shock_db, shock_kwargs) = ShockFunction.sneaky_db(out_db, db_end_of_loop, n_steps = 25, loop_name='oil_loop')

*3: Define the solution we wish to store for each iteration (conditions argument is optional, but here we extract only the production sector goods):*

In [133]:
shock_db.series["PwT_oil_loop_subset"].vals

Index(['C_EH_base_oil', 'C_EL_base_oil', 'C_ER_base_oil', 'ID_1_oil',
       'ID_2_oil', 'ID_5_oil', 'U_ID_C_EH_1_base_oil', 'U_ID_C_EH_2_base_oil',
       'U_ID_C_EH_3_base_oil', 'U_ID_C_EL_1_base_oil', 'U_ID_C_EL_2_base_oil',
       'U_ID_C_EL_3_base_oil', 'U_ID_C_EL_4_base_oil', 'U_ID_C_EL_5_base_oil',
       'U_ID_C_ER_1_base_oil', 'U_ID_C_ER_2_base_oil'],
      dtype='object', name='n')

In [134]:
store_sol = {'qD': {'domains': shock_kwargs['loop_name']}}

*4: Create Shock instance*

In [135]:
shock = mi.std_UEVAS_from_db(shock_db, loop_name=shock_kwargs['loop_name'], shock_name=shock_kwargs['shock_name'], store_sol=store_sol)
#shock = mi.std_UEVAS_from_db(shock_db, loop_name=shock_kwargs['loop_name'], shock_name=shock_kwargs['shock_name'])
#solve_sneakily skal have selve checkpointet
#Gør noget så sneaky solve laver et sidste checkpoint

*5: Execute shock from the 'calibrated' checkpoint i.e. the model state after the sigmas have been chosen:*

In [136]:
mi.execute_shock_from_cp(shock=shock, cp=m.checkpoints['calibrated'])

In [137]:
sol_qD = mi.out_db.series["sol_qD"].vals

In [138]:
#tech_names = ["ID_thighest", "ID_thigher", "ID_tmedium", "ID_tlower", "ID_tlowest"]
#sol_qD.loc(axis=0)["oil_loop_1", tech_names]

In [139]:
sol_qD.reset_index().to_excel(os.getcwd() + "\\oil_price_shock_" + inputfile.split("_")[-1], index=False)

In [140]:
#m.model_instances["oilshock"].solve_sneakily(db_star=db_end_of_loop, from_cp=True, cp_init=m.checkpoints["oilshock"])

The solution is stored as a parameter noted as "sol_"+ the name of the variable. In our case, the specific quantities are stored:

### **3.1 Diagnostics (dated)**
When the model cannot be solved, we run some diagnostics:

In [ ]:
# for symbol in ["PbT", "PwT", "qD"]:
#     start_vals = m.model_instances["baseline"].out_db.series["load_" + symbol].vals
#     end_vals = m.model_instances["baseline"].out_db.series[symbol].vals
#     rel_changes = (end_vals - start_vals)/start_vals
#     large_changes = rel_changes[~(rel_changes.between(-0.0001, 0.0001))]
#     if len(large_changes) > 0:
#         print("Large deviations for symbol \n" + symbol + ":")
#         print(large_changes, "\n")
#     else:
#         print("No large deviations for symbol: " + symbol + ".\n")

In [ ]:
# from decimal import Decimal

In [ ]:
# #Demands
# print(Decimal(qD["U_EOP_tau1_1"] - mu[("U_EOP_tau1_1", "C_CO2_1")] * qS["C_CO2_1"]))
# print(Decimal(qD["EOP_tau1"] - mu.loc[("U_EOP_tau1_1", "EOP_tau1")] * qD["U_EOP_tau1_1"]))
# print(Decimal(qD["EOP_tau1_electricity"] - mu[("EOP_tau1_electricity", "EOP_tau1")] * qD["EOP_tau1"]))
# print(Decimal(qD["EOP_tau1_K"] - mu[("EOP_tau1_K", "EOP_tau1")] * qD["EOP_tau1"]))
# #zero profits
# print(Decimal(qS["C_CO2_1"]*PbT["C_CO2_1"] - qD["U_EOP_tau1_1"]*PwT["U_EOP_tau1_1"]))
# print(Decimal(qD["U_EOP_tau1_1"]*PwT["U_EOP_tau1_1"] - qD["EOP_tau1"]*PwT["EOP_tau1"]))
# print(Decimal(qD["EOP_tau1"]*PwT["EOP_tau1"] - qD["EOP_tau1_electricity"]*PwT["EOP_tau1_electricity"] - qD["EOP_tau1_K"]*PwT["EOP_tau1_K"]))